In [1]:
import os
import tensorflow as tf
import pandas as pd
import json
import numpy as np

from pandas.io.json import json_normalize #package for flattening json in pandas df


/home/student/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
!ls


preprocessed_test_set_without_hits.pkl	 test_v2.csv
preprocessed_train_set_without_hits.pkl  train_v2.csv
sample_submission_v2.csv		 Untitled.ipynb
test_set_hits_column.pkl


In [3]:
one_row = pd.read_csv("train_v2.csv", nrows=1)

In [4]:
one_row = list(one_row)

In [5]:
one_row

['channelGrouping',
 'customDimensions',
 'date',
 'device',
 'fullVisitorId',
 'geoNetwork',
 'hits',
 'socialEngagementType',
 'totals',
 'trafficSource',
 'visitId',
 'visitNumber',
 'visitStartTime']

In [21]:
%%time 
no_of_rows = 803178
# converter applies a converter function while you are loading the data, you are loading it as
# json.loadstring = json.loads() for each of those columns in the list
train_set = pd.read_csv("train_v2.csv", usecols=[i for i in one_row if i!= 'hits'], dtype={'fullVisitorId': 'str'}, nrows = no_of_rows)

CPU times: user 19.3 s, sys: 2.4 s, total: 21.7 s
Wall time: 27.4 s


In [22]:
train_set.head()

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",3162355547410993243,"{""continent"": ""Europe"", ""subContinent"": ""Weste...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508198450,1,1508198450
1,Referral,"[{'index': '4', 'value': 'North America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",8934116514970143966,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""referralPath"": ""/a/google.com/transportation...",1508176307,6,1508176307
2,Direct,"[{'index': '4', 'value': 'North America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",7992466427990357681,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""(direct)""...",1508201613,1,1508201613
3,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",9075655783635761930,"{""continent"": ""Asia"", ""subContinent"": ""Western...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508169851,1,1508169851
4,Organic Search,"[{'index': '4', 'value': 'Central America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",6960673291025684308,"{""continent"": ""Americas"", ""subContinent"": ""Cen...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508190552,1,1508190552


In [23]:
train_set.shape

(803178, 12)

In [10]:
test_set = pd.read_pickle("preprocessed_test_set_without_hits.pkl")

In [11]:
test_set.shape

(401589, 53)

In [10]:
train_set["device"][0]

'{"browser": "Firefox", "browserVersion": "not available in demo dataset", "browserSize": "not available in demo dataset", "operatingSystem": "Windows", "operatingSystemVersion": "not available in demo dataset", "isMobile": false, "mobileDeviceBranding": "not available in demo dataset", "mobileDeviceModel": "not available in demo dataset", "mobileInputSelector": "not available in demo dataset", "mobileDeviceInfo": "not available in demo dataset", "mobileDeviceMarketingName": "not available in demo dataset", "flashVersion": "not available in demo dataset", "language": "not available in demo dataset", "screenColors": "not available in demo dataset", "screenResolution": "not available in demo dataset", "deviceCategory": "desktop"}'

In [11]:
type(train_set["device"][0])

str

In [12]:
train_set.dtypes

channelGrouping         object
customDimensions        object
date                     int64
device                  object
fullVisitorId           object
geoNetwork              object
socialEngagementType    object
totals                  object
trafficSource           object
visitId                  int64
visitNumber              int64
visitStartTime           int64
dtype: object

In [13]:
type(train_set["customDimensions"][0])

str

In [14]:
train_set["customDimensions"][0]

"[{'index': '4', 'value': 'EMEA'}]"

In [24]:
train_set = train_set.join(train_set["device"].apply(json.loads).apply(pd.Series))

In [25]:
train_set.head()

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,socialEngagementType,totals,trafficSource,visitId,...,mobileDeviceBranding,mobileDeviceModel,mobileInputSelector,mobileDeviceInfo,mobileDeviceMarketingName,flashVersion,language,screenColors,screenResolution,deviceCategory
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",3162355547410993243,"{""continent"": ""Europe"", ""subContinent"": ""Weste...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508198450,...,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop
1,Referral,"[{'index': '4', 'value': 'North America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",8934116514970143966,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""referralPath"": ""/a/google.com/transportation...",1508176307,...,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop
2,Direct,"[{'index': '4', 'value': 'North America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",7992466427990357681,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""(direct)""...",1508201613,...,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,mobile
3,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",9075655783635761930,"{""continent"": ""Asia"", ""subContinent"": ""Western...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508169851,...,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop
4,Organic Search,"[{'index': '4', 'value': 'Central America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",6960673291025684308,"{""continent"": ""Americas"", ""subContinent"": ""Cen...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508190552,...,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop


In [26]:
%%time 
train_set = train_set.join(train_set["geoNetwork"].apply(json.loads).apply(pd.Series))

CPU times: user 2min 35s, sys: 727 ms, total: 2min 36s
Wall time: 2min 35s


In [27]:
train_set.head()

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,socialEngagementType,totals,trafficSource,visitId,...,subContinent,country,region,metro,city,cityId,networkDomain,latitude,longitude,networkLocation
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",3162355547410993243,"{""continent"": ""Europe"", ""subContinent"": ""Weste...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508198450,...,Western Europe,Germany,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset
1,Referral,"[{'index': '4', 'value': 'North America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",8934116514970143966,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""referralPath"": ""/a/google.com/transportation...",1508176307,...,Northern America,United States,California,San Francisco-Oakland-San Jose CA,Cupertino,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset
2,Direct,"[{'index': '4', 'value': 'North America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",7992466427990357681,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""(direct)""...",1508201613,...,Northern America,United States,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,windjammercable.net,not available in demo dataset,not available in demo dataset,not available in demo dataset
3,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",9075655783635761930,"{""continent"": ""Asia"", ""subContinent"": ""Western...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508169851,...,Western Asia,Turkey,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,unknown.unknown,not available in demo dataset,not available in demo dataset,not available in demo dataset
4,Organic Search,"[{'index': '4', 'value': 'Central America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",6960673291025684308,"{""continent"": ""Americas"", ""subContinent"": ""Cen...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508190552,...,Central America,Mexico,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,prod-infinitum.com.mx,not available in demo dataset,not available in demo dataset,not available in demo dataset


In [16]:
original_hits_column_train = train_set["hits"]

In [17]:
original_hits_column_train[0]

"[{'hitNumber': '1', 'time': '0', 'hour': '21', 'minute': '29', 'isInteraction': True, 'page': {'pagePath': '/home', 'hostname': 'shop.googlemerchandisestore.com', 'pageTitle': 'Home', 'pagePathLevel1': '/home', 'pagePathLevel2': '', 'pagePathLevel3': '', 'pagePathLevel4': ''}, 'appInfo': {'screenName': 'shop.googlemerchandisestore.com/home', 'landingScreenName': 'shop.googlemerchandisestore.com/home', 'exitScreenName': 'shop.googlemerchandisestore.com/home', 'screenDepth': '0'}, 'exceptionInfo': {'isFatal': True}, 'eventInfo': {'eventCategory': 'Enhanced Ecommerce', 'eventAction': 'Promotion Click'}, 'product': [], 'promotion': [{'promoId': 'Category Row 2', 'promoName': 'Accessories', 'promoCreative': 'toy.png', 'promoPosition': 'Category Row 2'}], 'promotionActionInfo': {'promoIsClick': True}, 'eCommerceAction': {'action_type': '0', 'step': '1'}, 'experiment': [], 'customVariables': [], 'customDimensions': [], 'customMetrics': [], 'type': 'EVENT', 'social': {'socialNetwork': '(not s

In [18]:
test_set.drop(["hits"], axis=1, inplace=True)

In [19]:
test_set.head()

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,socialEngagementType,totals,trafficSource,visitId,...,subContinent,country,region,metro,city,cityId,networkDomain,latitude,longitude,networkLocation
0,Organic Search,"[{'index': '4', 'value': 'APAC'}]",20180511,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",7460955084541987166,"{""continent"": ""Asia"", ""subContinent"": ""Souther...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""4"", ""pageviews"": ""3"",...","{""referralPath"": ""(not set)"", ""campaign"": ""(no...",1526099341,...,Southern Asia,India,Delhi,(not set),(not set),not available in demo dataset,unknown.unknown,not available in demo dataset,not available in demo dataset,not available in demo dataset
1,Direct,"[{'index': '4', 'value': 'North America'}]",20180511,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",460252456180441002,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""4"", ""pageviews"": ""3"",...","{""referralPath"": ""(not set)"", ""campaign"": ""(no...",1526064483,...,Northern America,United States,California,San Francisco-Oakland-San Jose CA,San Francisco,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset
2,Organic Search,"[{'index': '4', 'value': 'North America'}]",20180511,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",3461808543879602873,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""4"", ""pageviews"": ""3"",...","{""referralPath"": ""(not set)"", ""campaign"": ""(no...",1526067157,...,Northern America,United States,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,onlinecomputerworks.com,not available in demo dataset,not available in demo dataset,not available in demo dataset
3,Direct,"[{'index': '4', 'value': 'North America'}]",20180511,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",975129477712150630,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""5"", ""pageviews"": ""4"",...","{""referralPath"": ""(not set)"", ""campaign"": ""(no...",1526107551,...,Northern America,United States,Texas,Houston TX,Houston,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset
4,Organic Search,"[{'index': '4', 'value': 'North America'}]",20180511,"{""browser"": ""Internet Explorer"", ""browserVersi...",8381672768065729990,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""5"", ""pageviews"": ""4"",...","{""referralPath"": ""(not set)"", ""campaign"": ""(no...",1526060254,...,Northern America,United States,California,Los Angeles CA,Irvine,not available in demo dataset,com,not available in demo dataset,not available in demo dataset,not available in demo dataset


In [28]:
train_set = train_set.join(train_set["totals"].apply(json.loads).apply(pd.Series))

In [29]:
train_set.head()

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,socialEngagementType,totals,trafficSource,visitId,...,visits,hits,pageviews,bounces,newVisits,sessionQualityDim,timeOnSite,transactions,transactionRevenue,totalTransactionRevenue
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",3162355547410993243,"{""continent"": ""Europe"", ""subContinent"": ""Weste...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508198450,...,1,1,1,1,1,1,NaN,NaN,NaN,NaN
1,Referral,"[{'index': '4', 'value': 'North America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",8934116514970143966,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""referralPath"": ""/a/google.com/transportation...",1508176307,...,1,2,2,NaN,NaN,2,28,NaN,NaN,NaN
2,Direct,"[{'index': '4', 'value': 'North America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",7992466427990357681,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""(direct)""...",1508201613,...,1,2,2,NaN,1,1,38,NaN,NaN,NaN
3,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",9075655783635761930,"{""continent"": ""Asia"", ""subContinent"": ""Western...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508169851,...,1,2,2,NaN,1,1,1,NaN,NaN,NaN
4,Organic Search,"[{'index': '4', 'value': 'Central America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",6960673291025684308,"{""continent"": ""Americas"", ""subContinent"": ""Cen...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508190552,...,1,2,2,NaN,1,1,52,NaN,NaN,NaN


In [30]:
%%time 
train_set = train_set.join(train_set["trafficSource"].apply(json.loads).apply(pd.Series))

CPU times: user 2min 35s, sys: 731 ms, total: 2min 36s
Wall time: 2min 35s


In [31]:
train_set.head()

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,socialEngagementType,totals,trafficSource,visitId,...,totalTransactionRevenue,campaign,source,medium,keyword,adwordsClickInfo,referralPath,isTrueDirect,adContent,campaignCode
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",3162355547410993243,"{""continent"": ""Europe"", ""subContinent"": ""Weste...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508198450,...,NaN,(not set),google,organic,water bottle,{'criteriaParameters': 'not available in demo ...,NaN,NaN,NaN,NaN
1,Referral,"[{'index': '4', 'value': 'North America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",8934116514970143966,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""referralPath"": ""/a/google.com/transportation...",1508176307,...,NaN,(not set),sites.google.com,referral,NaN,{'criteriaParameters': 'not available in demo ...,/a/google.com/transportation/mtv-services/bike...,NaN,NaN,NaN
2,Direct,"[{'index': '4', 'value': 'North America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",7992466427990357681,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""(direct)""...",1508201613,...,NaN,(not set),(direct),(none),NaN,{'criteriaParameters': 'not available in demo ...,NaN,True,NaN,NaN
3,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",9075655783635761930,"{""continent"": ""Asia"", ""subContinent"": ""Western...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508169851,...,NaN,(not set),google,organic,(not provided),{'criteriaParameters': 'not available in demo ...,NaN,NaN,NaN,NaN
4,Organic Search,"[{'index': '4', 'value': 'Central America'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",6960673291025684308,"{""continent"": ""Americas"", ""subContinent"": ""Cen...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508190552,...,NaN,(not set),google,organic,(not provided),{'criteriaParameters': 'not available in demo ...,NaN,NaN,NaN,NaN


In [32]:
train_set.drop(["device", "geoNetwork", "totals", "trafficSource"], axis=1, inplace=True)

In [33]:
train_set.head()

,channelGrouping,customDimensions,date,fullVisitorId,socialEngagementType,visitId,visitNumber,visitStartTime,browser,browserVersion,...,totalTransactionRevenue,campaign,source,medium,keyword,adwordsClickInfo,referralPath,isTrueDirect,adContent,campaignCode
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,3162355547410993243,Not Socially Engaged,1508198450,1,1508198450,Firefox,not available in demo dataset,...,NaN,(not set),google,organic,water bottle,{'criteriaParameters': 'not available in demo ...,NaN,NaN,NaN,NaN
1,Referral,"[{'index': '4', 'value': 'North America'}]",20171016,8934116514970143966,Not Socially Engaged,1508176307,6,1508176307,Chrome,not available in demo dataset,...,NaN,(not set),sites.google.com,referral,NaN,{'criteriaParameters': 'not available in demo ...,/a/google.com/transportation/mtv-services/bike...,NaN,NaN,NaN
2,Direct,"[{'index': '4', 'value': 'North America'}]",20171016,7992466427990357681,Not Socially Engaged,1508201613,1,1508201613,Chrome,not available in demo dataset,...,NaN,(not set),(direct),(none),NaN,{'criteriaParameters': 'not available in demo ...,NaN,True,NaN,NaN
3,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,9075655783635761930,Not Socially Engaged,1508169851,1,1508169851,Chrome,not available in demo dataset,...,NaN,(not set),google,organic,(not provided),{'criteriaParameters': 'not available in demo ...,NaN,NaN,NaN,NaN
4,Organic Search,"[{'index': '4', 'value': 'Central America'}]",20171016,6960673291025684308,Not Socially Engaged,1508190552,1,1508190552,Chrome,not available in demo dataset,...,NaN,(not set),google,organic,(not provided),{'criteriaParameters': 'not available in demo ...,NaN,NaN,NaN,NaN


In [34]:
train_set.to_pickle("preprocessed_train_set_without_hits.pkl")

In [19]:
df = pd.read_pickle("preprocessed_train_set_without_hits.pkl")

In [36]:
df.head()

,channelGrouping,customDimensions,date,fullVisitorId,socialEngagementType,visitId,visitNumber,visitStartTime,browser,browserVersion,...,totalTransactionRevenue,campaign,source,medium,keyword,adwordsClickInfo,referralPath,isTrueDirect,adContent,campaignCode
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,3162355547410993243,Not Socially Engaged,1508198450,1,1508198450,Firefox,not available in demo dataset,...,NaN,(not set),google,organic,water bottle,{'criteriaParameters': 'not available in demo ...,NaN,NaN,NaN,NaN
1,Referral,"[{'index': '4', 'value': 'North America'}]",20171016,8934116514970143966,Not Socially Engaged,1508176307,6,1508176307,Chrome,not available in demo dataset,...,NaN,(not set),sites.google.com,referral,NaN,{'criteriaParameters': 'not available in demo ...,/a/google.com/transportation/mtv-services/bike...,NaN,NaN,NaN
2,Direct,"[{'index': '4', 'value': 'North America'}]",20171016,7992466427990357681,Not Socially Engaged,1508201613,1,1508201613,Chrome,not available in demo dataset,...,NaN,(not set),(direct),(none),NaN,{'criteriaParameters': 'not available in demo ...,NaN,True,NaN,NaN
3,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,9075655783635761930,Not Socially Engaged,1508169851,1,1508169851,Chrome,not available in demo dataset,...,NaN,(not set),google,organic,(not provided),{'criteriaParameters': 'not available in demo ...,NaN,NaN,NaN,NaN
4,Organic Search,"[{'index': '4', 'value': 'Central America'}]",20171016,6960673291025684308,Not Socially Engaged,1508190552,1,1508190552,Chrome,not available in demo dataset,...,NaN,(not set),google,organic,(not provided),{'criteriaParameters': 'not available in demo ...,NaN,NaN,NaN,NaN


In [42]:
one_row

['channelGrouping',
 'customDimensions',
 'date',
 'device',
 'fullVisitorId',
 'geoNetwork',
 'hits',
 'socialEngagementType',
 'totals',
 'trafficSource',
 'visitId',
 'visitNumber',
 'visitStartTime']

In [57]:
first_row = pd.read_csv("train_v2.csv", nrows=1, skiprows=no_of_rows, header=None, names=one_row)

In [58]:
first_row

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,hits,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",7779444718212527610,"{""continent"": ""Americas"", ""subContinent"": ""Nor...","[{'hitNumber': '1', 'time': '0', 'hour': '17',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""5"", ""pageviews"": ""5"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507250518,2,1507250518


In [6]:
%%time 
no_of_rows = 803178
# converter applies a converter function while you are loading the data, you are loading it as
# json.loadstring = json.loads() for each of those columns in the list
train_set = pd.read_csv("train_v2.csv",header=None, names=one_row, usecols=[i for i in one_row if i!= 'hits'], dtype={'fullVisitorId': 'str'}, skiprows= no_of_rows)

CPU times: user 33.1 s, sys: 4.48 s, total: 37.6 s
Wall time: 53.4 s


In [7]:
train_set.head()

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",7779444718212527610,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""5"", ""pageviews"": ""5"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507250518,2,1507250518
1,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Safari"", ""browserVersion"": ""not a...",8127326206655250634,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""6"", ""pageviews"": ""6"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507250122,1,1507250122
2,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",8512418246113365791,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""6"", ""pageviews"": ""6"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507237813,1,1507237813
3,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",4331867527339252882,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""6"", ""pageviews"": ""6"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507250491,1,1507250491
4,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Safari"", ""browserVersion"": ""not a...",6311336399415104146,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""8"", ""pageviews"": ""8"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507243409,1,1507243409


In [8]:
train_set.shape

(905160, 12)

In [9]:
%%time
train_set = train_set.join(train_set["device"].apply(json.loads).apply(pd.Series))

CPU times: user 2min 43s, sys: 2.53 s, total: 2min 46s
Wall time: 2min 46s


In [10]:
train_set.head()

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,socialEngagementType,totals,trafficSource,visitId,...,mobileDeviceBranding,mobileDeviceModel,mobileInputSelector,mobileDeviceInfo,mobileDeviceMarketingName,flashVersion,language,screenColors,screenResolution,deviceCategory
0,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",7779444718212527610,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""5"", ""pageviews"": ""5"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507250518,...,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,mobile
1,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Safari"", ""browserVersion"": ""not a...",8127326206655250634,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""6"", ""pageviews"": ""6"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507250122,...,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,tablet
2,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",8512418246113365791,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""6"", ""pageviews"": ""6"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507237813,...,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop
3,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",4331867527339252882,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""6"", ""pageviews"": ""6"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507250491,...,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,mobile
4,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Safari"", ""browserVersion"": ""not a...",6311336399415104146,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""8"", ""pageviews"": ""8"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507243409,...,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,mobile


In [11]:
%%time 
train_set = train_set.join(train_set["geoNetwork"].apply(json.loads).apply(pd.Series))

CPU times: user 2min 55s, sys: 778 ms, total: 2min 56s
Wall time: 2min 55s


In [12]:
train_set.head()

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,socialEngagementType,totals,trafficSource,visitId,...,subContinent,country,region,metro,city,cityId,networkDomain,latitude,longitude,networkLocation
0,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",7779444718212527610,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""5"", ""pageviews"": ""5"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507250518,...,Northern America,United States,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,spcsdns.net,not available in demo dataset,not available in demo dataset,not available in demo dataset
1,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Safari"", ""browserVersion"": ""not a...",8127326206655250634,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""6"", ""pageviews"": ""6"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507250122,...,Northern America,United States,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset
2,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",8512418246113365791,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""6"", ""pageviews"": ""6"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507237813,...,Northern America,Canada,British Columbia,(not set),Vancouver,not available in demo dataset,bigpipeinc.com,not available in demo dataset,not available in demo dataset,not available in demo dataset
3,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",4331867527339252882,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""6"", ""pageviews"": ""6"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507250491,...,Northern America,United States,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset
4,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Safari"", ""browserVersion"": ""not a...",6311336399415104146,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""8"", ""pageviews"": ""8"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507243409,...,Northern America,United States,California,San Francisco-Oakland-San Jose CA,Sunnyvale,not available in demo dataset,juniper.net,not available in demo dataset,not available in demo dataset,not available in demo dataset


In [13]:
%%time
train_set = train_set.join(train_set["totals"].apply(json.loads).apply(pd.Series))

CPU times: user 2min 44s, sys: 751 ms, total: 2min 45s
Wall time: 2min 44s


In [14]:
train_set.head()

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,socialEngagementType,totals,trafficSource,visitId,...,visits,hits,pageviews,timeOnSite,sessionQualityDim,newVisits,transactions,transactionRevenue,totalTransactionRevenue,bounces
0,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",7779444718212527610,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""5"", ""pageviews"": ""5"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507250518,...,1,5,5,170,1,NaN,NaN,NaN,NaN,NaN
1,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Safari"", ""browserVersion"": ""not a...",8127326206655250634,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""6"", ""pageviews"": ""6"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507250122,...,1,6,6,190,1,1,NaN,NaN,NaN,NaN
2,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",8512418246113365791,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""6"", ""pageviews"": ""6"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507237813,...,1,6,6,20,1,1,NaN,NaN,NaN,NaN
3,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",4331867527339252882,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""6"", ""pageviews"": ""6"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507250491,...,1,6,6,161,1,1,NaN,NaN,NaN,NaN
4,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Safari"", ""browserVersion"": ""not a...",6311336399415104146,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""8"", ""pageviews"": ""8"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507243409,...,1,8,8,270,2,1,NaN,NaN,NaN,NaN


In [15]:
%%time 
train_set = train_set.join(train_set["trafficSource"].apply(json.loads).apply(pd.Series))

CPU times: user 2min 50s, sys: 832 ms, total: 2min 50s
Wall time: 2min 50s


In [16]:
train_set.head()

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,socialEngagementType,totals,trafficSource,visitId,...,totalTransactionRevenue,bounces,campaign,source,medium,keyword,adwordsClickInfo,isTrueDirect,referralPath,adContent
0,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",7779444718212527610,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""5"", ""pageviews"": ""5"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507250518,...,NaN,NaN,(not set),google,organic,(not provided),{'criteriaParameters': 'not available in demo ...,True,NaN,NaN
1,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Safari"", ""browserVersion"": ""not a...",8127326206655250634,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""6"", ""pageviews"": ""6"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507250122,...,NaN,NaN,(not set),google,organic,(not provided),{'criteriaParameters': 'not available in demo ...,NaN,NaN,NaN
2,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",8512418246113365791,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""6"", ""pageviews"": ""6"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507237813,...,NaN,NaN,(not set),google,organic,(not provided),{'criteriaParameters': 'not available in demo ...,NaN,NaN,NaN
3,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",4331867527339252882,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""6"", ""pageviews"": ""6"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507250491,...,NaN,NaN,(not set),google,organic,(not provided),{'criteriaParameters': 'not available in demo ...,NaN,NaN,NaN
4,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,"{""browser"": ""Safari"", ""browserVersion"": ""not a...",6311336399415104146,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""8"", ""pageviews"": ""8"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1507243409,...,NaN,NaN,(not set),google,organic,(not provided),{'criteriaParameters': 'not available in demo ...,NaN,NaN,NaN


In [17]:
train_set.drop(["device", "geoNetwork", "totals", "trafficSource"], axis=1, inplace=True)

In [18]:
train_set.head()

,channelGrouping,customDimensions,date,fullVisitorId,socialEngagementType,visitId,visitNumber,visitStartTime,browser,browserVersion,...,totalTransactionRevenue,bounces,campaign,source,medium,keyword,adwordsClickInfo,isTrueDirect,referralPath,adContent
0,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,7779444718212527610,Not Socially Engaged,1507250518,2,1507250518,Chrome,not available in demo dataset,...,NaN,NaN,(not set),google,organic,(not provided),{'criteriaParameters': 'not available in demo ...,True,NaN,NaN
1,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,8127326206655250634,Not Socially Engaged,1507250122,1,1507250122,Safari,not available in demo dataset,...,NaN,NaN,(not set),google,organic,(not provided),{'criteriaParameters': 'not available in demo ...,NaN,NaN,NaN
2,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,8512418246113365791,Not Socially Engaged,1507237813,1,1507237813,Chrome,not available in demo dataset,...,NaN,NaN,(not set),google,organic,(not provided),{'criteriaParameters': 'not available in demo ...,NaN,NaN,NaN
3,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,4331867527339252882,Not Socially Engaged,1507250491,1,1507250491,Chrome,not available in demo dataset,...,NaN,NaN,(not set),google,organic,(not provided),{'criteriaParameters': 'not available in demo ...,NaN,NaN,NaN
4,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171005,6311336399415104146,Not Socially Engaged,1507243409,1,1507243409,Safari,not available in demo dataset,...,NaN,NaN,(not set),google,organic,(not provided),{'criteriaParameters': 'not available in demo ...,NaN,NaN,NaN


In [20]:
train_set.to_pickle("preprocessed_train_set_without_hits2.pkl")

In [21]:
df2 = pd.read_pickle("preprocessed_train_set_without_hits2.pkl")

In [22]:
df.keys()

Index(['channelGrouping', 'customDimensions', 'date', 'fullVisitorId',
       'socialEngagementType', 'visitId', 'visitNumber', 'visitStartTime',
       'browser', 'browserVersion', 'browserSize', 'operatingSystem',
       'operatingSystemVersion', 'isMobile', 'mobileDeviceBranding',
       'mobileDeviceModel', 'mobileInputSelector', 'mobileDeviceInfo',
       'mobileDeviceMarketingName', 'flashVersion', 'language', 'screenColors',
       'screenResolution', 'deviceCategory', 'continent', 'subContinent',
       'country', 'region', 'metro', 'city', 'cityId', 'networkDomain',
       'latitude', 'longitude', 'networkLocation', 'visits', 'hits',
       'pageviews', 'bounces', 'newVisits', 'sessionQualityDim', 'timeOnSite',
       'transactions', 'transactionRevenue', 'totalTransactionRevenue',
       'campaign', 'source', 'medium', 'keyword', 'adwordsClickInfo',
       'referralPath', 'isTrueDirect', 'adContent', 'campaignCode'],
      dtype='object')

In [23]:
df2.keys()

Index(['channelGrouping', 'customDimensions', 'date', 'fullVisitorId',
       'socialEngagementType', 'visitId', 'visitNumber', 'visitStartTime',
       'browser', 'browserVersion', 'browserSize', 'operatingSystem',
       'operatingSystemVersion', 'isMobile', 'mobileDeviceBranding',
       'mobileDeviceModel', 'mobileInputSelector', 'mobileDeviceInfo',
       'mobileDeviceMarketingName', 'flashVersion', 'language', 'screenColors',
       'screenResolution', 'deviceCategory', 'continent', 'subContinent',
       'country', 'region', 'metro', 'city', 'cityId', 'networkDomain',
       'latitude', 'longitude', 'networkLocation', 'visits', 'hits',
       'pageviews', 'timeOnSite', 'sessionQualityDim', 'newVisits',
       'transactions', 'transactionRevenue', 'totalTransactionRevenue',
       'bounces', 'campaign', 'source', 'medium', 'keyword',
       'adwordsClickInfo', 'isTrueDirect', 'referralPath', 'adContent'],
      dtype='object')

In [25]:
df.keys() == df2.keys()

ValueError: Lengths must match to compare

In [28]:
df2 = list(df2.keys())

In [32]:
df2

['channelGrouping',
 'customDimensions',
 'date',
 'fullVisitorId',
 'socialEngagementType',
 'visitId',
 'visitNumber',
 'visitStartTime',
 'browser',
 'browserVersion',
 'browserSize',
 'operatingSystem',
 'operatingSystemVersion',
 'isMobile',
 'mobileDeviceBranding',
 'mobileDeviceModel',
 'mobileInputSelector',
 'mobileDeviceInfo',
 'mobileDeviceMarketingName',
 'flashVersion',
 'language',
 'screenColors',
 'screenResolution',
 'deviceCategory',
 'continent',
 'subContinent',
 'country',
 'region',
 'metro',
 'city',
 'cityId',
 'networkDomain',
 'latitude',
 'longitude',
 'networkLocation',
 'visits',
 'hits',
 'pageviews',
 'timeOnSite',
 'sessionQualityDim',
 'newVisits',
 'transactions',
 'transactionRevenue',
 'totalTransactionRevenue',
 'bounces',
 'campaign',
 'source',
 'medium',
 'keyword',
 'adwordsClickInfo',
 'isTrueDirect',
 'referralPath',
 'adContent']

In [34]:
df = list(df.keys())

In [33]:
list(df.keys()) == df2[:-1]

False

In [35]:
for one, two in zip(df, df2):
    if one != two:
        print(one, two)

bounces timeOnSite
newVisits sessionQualityDim
sessionQualityDim newVisits
timeOnSite transactions
transactions transactionRevenue
transactionRevenue totalTransactionRevenue
totalTransactionRevenue bounces
referralPath isTrueDirect
isTrueDirect referralPath


In [44]:
df = sorted(df)

In [45]:
df2 = sorted(df2)

In [46]:
for one, two in zip(df, df2):
    if one != two:
        print(one, two)

campaignCode channelGrouping
channelGrouping city
city cityId
cityId continent
continent country
country customDimensions
customDimensions date
date deviceCategory
deviceCategory flashVersion
flashVersion fullVisitorId
fullVisitorId hits
hits isMobile
isMobile isTrueDirect
isTrueDirect keyword
keyword language
language latitude
latitude longitude
longitude medium
medium metro
metro mobileDeviceBranding
mobileDeviceBranding mobileDeviceInfo
mobileDeviceInfo mobileDeviceMarketingName
mobileDeviceMarketingName mobileDeviceModel
mobileDeviceModel mobileInputSelector
mobileInputSelector networkDomain
networkDomain networkLocation
networkLocation newVisits
newVisits operatingSystem
operatingSystem operatingSystemVersion
operatingSystemVersion pageviews
pageviews referralPath
referralPath region
region screenColors
screenColors screenResolution
screenResolution sessionQualityDim
sessionQualityDim socialEngagementType
socialEngagementType source
source subContinent
subContinent timeOnSite
time

In [49]:
df2

['adContent',
 'adwordsClickInfo',
 'bounces',
 'browser',
 'browserSize',
 'browserVersion',
 'campaign',
 'channelGrouping',
 'city',
 'cityId',
 'continent',
 'country',
 'customDimensions',
 'date',
 'deviceCategory',
 'flashVersion',
 'fullVisitorId',
 'hits',
 'isMobile',
 'isTrueDirect',
 'keyword',
 'language',
 'latitude',
 'longitude',
 'medium',
 'metro',
 'mobileDeviceBranding',
 'mobileDeviceInfo',
 'mobileDeviceMarketingName',
 'mobileDeviceModel',
 'mobileInputSelector',
 'networkDomain',
 'networkLocation',
 'newVisits',
 'operatingSystem',
 'operatingSystemVersion',
 'pageviews',
 'referralPath',
 'region',
 'screenColors',
 'screenResolution',
 'sessionQualityDim',
 'socialEngagementType',
 'source',
 'subContinent',
 'timeOnSite',
 'totalTransactionRevenue',
 'transactionRevenue',
 'transactions',
 'visitId',
 'visitNumber',
 'visitStartTime',
 'visits']

In [48]:
df

['adContent',
 'adwordsClickInfo',
 'bounces',
 'browser',
 'browserSize',
 'browserVersion',
 'campaign',
 'campaignCode',
 'channelGrouping',
 'city',
 'cityId',
 'continent',
 'country',
 'customDimensions',
 'date',
 'deviceCategory',
 'flashVersion',
 'fullVisitorId',
 'hits',
 'isMobile',
 'isTrueDirect',
 'keyword',
 'language',
 'latitude',
 'longitude',
 'medium',
 'metro',
 'mobileDeviceBranding',
 'mobileDeviceInfo',
 'mobileDeviceMarketingName',
 'mobileDeviceModel',
 'mobileInputSelector',
 'networkDomain',
 'networkLocation',
 'newVisits',
 'operatingSystem',
 'operatingSystemVersion',
 'pageviews',
 'referralPath',
 'region',
 'screenColors',
 'screenResolution',
 'sessionQualityDim',
 'socialEngagementType',
 'source',
 'subContinent',
 'timeOnSite',
 'totalTransactionRevenue',
 'transactionRevenue',
 'transactions',
 'visitId',
 'visitNumber',
 'visitStartTime',
 'visits']

In [40]:
d = {}
for one in df:
    d[one] = 1

In [41]:
d2 = {}
for two in df2:
    d2[two] = 1

In [42]:
d.keys()

dict_keys(['channelGrouping', 'customDimensions', 'date', 'fullVisitorId', 'socialEngagementType', 'visitId', 'visitNumber', 'visitStartTime', 'browser', 'browserVersion', 'browserSize', 'operatingSystem', 'operatingSystemVersion', 'isMobile', 'mobileDeviceBranding', 'mobileDeviceModel', 'mobileInputSelector', 'mobileDeviceInfo', 'mobileDeviceMarketingName', 'flashVersion', 'language', 'screenColors', 'screenResolution', 'deviceCategory', 'continent', 'subContinent', 'country', 'region', 'metro', 'city', 'cityId', 'networkDomain', 'latitude', 'longitude', 'networkLocation', 'visits', 'hits', 'pageviews', 'bounces', 'newVisits', 'sessionQualityDim', 'timeOnSite', 'transactions', 'transactionRevenue', 'totalTransactionRevenue', 'campaign', 'source', 'medium', 'keyword', 'adwordsClickInfo', 'referralPath', 'isTrueDirect', 'adContent', 'campaignCode'])

In [43]:
d2.keys()

dict_keys(['channelGrouping', 'customDimensions', 'date', 'fullVisitorId', 'socialEngagementType', 'visitId', 'visitNumber', 'visitStartTime', 'browser', 'browserVersion', 'browserSize', 'operatingSystem', 'operatingSystemVersion', 'isMobile', 'mobileDeviceBranding', 'mobileDeviceModel', 'mobileInputSelector', 'mobileDeviceInfo', 'mobileDeviceMarketingName', 'flashVersion', 'language', 'screenColors', 'screenResolution', 'deviceCategory', 'continent', 'subContinent', 'country', 'region', 'metro', 'city', 'cityId', 'networkDomain', 'latitude', 'longitude', 'networkLocation', 'visits', 'hits', 'pageviews', 'timeOnSite', 'sessionQualityDim', 'newVisits', 'transactions', 'transactionRevenue', 'totalTransactionRevenue', 'bounces', 'campaign', 'source', 'medium', 'keyword', 'adwordsClickInfo', 'isTrueDirect', 'referralPath', 'adContent'])

In [50]:
df= pd.read_pickle("preprocessed_train_set_without_hits.pkl")

In [54]:
df.drop(["campaignCode"], axis=1, inplace=True)

In [55]:
df.to_pickle("preprocessed_train_set_without_hits.pkl")

In [60]:
tf = pd.read_pickle("preprocessed_test_set_without_hits.pkl")

In [61]:
tf = sorted(list(tf.keys()))

In [ ]:
original_hits_column.to_pickle("test_set_hits_column.pkl")